In [1]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2MB 24kB/s 
     |████████████████████████████████| 3.0MB 46.2MB/s 
     |████████████████████████████████| 460kB 57.8MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
# Check Tensorflow version
tf.__version__

'2.2.0'

This is an example Colab that you may use to check the validity of your submission for the PGDL competition.

First we define the functions that we use for loading the sequential models.

## Utilities

In [4]:
class Sequential(tf.keras.Sequential):
  def __call__(self, x, tape=False, *args, **kwargs):
    if tape:
      tape.watch(x)
    return super(Sequential, self).__call__(x, *args, **kwargs)


def wrap_layer(layer_cls, *args, **kwargs):
    """Wraps a layer for computing the jacobian wrt to intermediate layers."""
    class wrapped_layer(layer_cls):
        def __call__(self, x, *args, **kwargs):
            self._last_seen_input = x
            return super(wrapped_layer, self).__call__(x, *args, **kwargs)
    return wrapped_layer(*args, **kwargs)


def model_def_to_keras_sequential(model_def):
    """Convert a model json to a Keras Sequential model.

    Args:
        model_def: A list of dictionaries, where each dict describes a layer to add
            to the model.

    Returns:
        A Keras Sequential model with the required architecture.
    """

    def _cast_to_integer_if_possible(dct):
        dct = dict(dct)
        for k, v in dct.items():
            if isinstance(v, float) and v.is_integer():
                dct[k] = int(v)
        return dct

    def parse_layer(layer_def):
        layer_cls = getattr(tf.keras.layers, layer_def['layer_name'])
        # layer_cls = wrap_layer(layer_cls)
        kwargs = dict(layer_def)
        del kwargs['layer_name']
        return wrap_layer(layer_cls, **_cast_to_integer_if_possible(kwargs))
        # return layer_cls(**_cast_to_integer_if_possible(kwargs))

    return Sequential([parse_layer(l) for l in model_def])


@tf.function()
def get_jacobian(model, inputs):
    """Get jacobians with respect to intermediate layers."""
    with tf.GradientTape(persistent=True) as tape:
        out = model(inputs, tape=tape)
    dct = {}
    for i, l in enumerate(model.layers):
        try:
            dct[i] = tape.batch_jacobian(out, l._last_seen_input)
        except AttributeError:  # no _last_seen_input, layer not wrapped (ex: flatten)
            dct[i] = None
    return dct

## Defining Dataset and Model

**Load Dataset**

We first load a example dataset (CIFAR-10).

In [5]:
ds = tfds.load('cifar10', split='train', shuffle_files=True)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteJGSYTT/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteJGSYTT/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.


In [6]:
ds_test = tfds.load('cifar10',split='test',shuffle_files=True)

**Make a model**

We create a simpel convolutional neural network, with the special model interface we made for the PGDL competition.

In [34]:
model = Sequential(
    [
     wrap_layer(tf.keras.layers.Conv2D, filters=16, kernel_size=5, strides=2, input_shape=(32, 32, 3), activation='relu'),
     wrap_layer(tf.keras.layers.MaxPool2D),
     wrap_layer(tf.keras.layers.Conv2D, filters=32, kernel_size=3, strides=2, activation='relu'),
     #wrap_layer(tf.keras.layers.MaxPool2D),
     wrap_layer(tf.keras.layers.Conv2D, filters=32, kernel_size=3, strides=2, activation='relu',padding='same'),
     wrap_layer(tf.keras.layers.Flatten),
     wrap_layer(tf.keras.layers.Dense, units=120, activation='relu'),
     wrap_layer(tf.keras.layers.Dense, units=84, activation='relu'),
     wrap_layer(tf.keras.layers.Dense, units=10)
    ]
)

Since this model is not trained, we will manually set its initial weights.

In [23]:
model.initial_weights = model.get_weights()#[np.ones(w.shape)*2 for w in model.get_weights()]

In [25]:
# save initial weights
model.save_weights("initial_weights.hdf5")

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
wrapped_layer_8 (wrapped_lay (None, 14, 14, 16)        1216      
_________________________________________________________________
wrapped_layer_9 (wrapped_lay (None, 7, 7, 16)          0         
_________________________________________________________________
wrapped_layer_10 (wrapped_la (None, 3, 3, 32)          4640      
_________________________________________________________________
wrapped_layer_11 (wrapped_la (None, 2, 2, 32)          9248      
_________________________________________________________________
wrapped_layer_12 (wrapped_la (None, 128)               0         
_________________________________________________________________
wrapped_layer_13 (wrapped_la (None, 120)               15480     
_________________________________________________________________
wrapped_layer_14 (wrapped_la (None, 84)               

# Training a model

In [15]:
iterator = iter(ds.batch(256))
x =[]
y =[]
for i in range(196):
  data = next(iterator)
  x.append(data['image'])
  y.append(data['label'])
x = np.concatenate(x,axis=0)
y = np.concatenate(y,axis=0) 
x = x/255
one_hot = np.zeros((len(y),10))
one_hot[np.arange(len(y)),y] = 1
y= one_hot

In [16]:
opt = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
model.fit(x,y,batch_size=128,epochs=100)

Epoch 1/100
391/391 [==============================] - 2s 4ms/step - loss: 1.8126 - accuracy: 0.3307
Epoch 2/100
391/391 [==============================] - 1s 4ms/step - loss: 1.5042 - accuracy: 0.4503
Epoch 3/100
391/391 [==============================] - 1s 4ms/step - loss: 1.3921 - accuracy: 0.4980
Epoch 4/100
391/391 [==============================] - 1s 4ms/step - loss: 1.3142 - accuracy: 0.5317
Epoch 5/100
391/391 [==============================] - 2s 4ms/step - loss: 1.2430 - accuracy: 0.5582
Epoch 6/100
391/391 [==============================] - 1s 4ms/step - loss: 1.1921 - accuracy: 0.5762
Epoch 7/100
391/391 [==============================] - 1s 4ms/step - loss: 1.1535 - accuracy: 0.5917
Epoch 8/100
391/391 [==============================] - 1s 4ms/step - loss: 1.1140 - accuracy: 0.6036
Epoch 9/100
391/391 [==============================] - 1s 4ms/step - loss: 1.0845 - accuracy: 0.6150
Epoch 10/100
391/391 [==============================] - 1s 4ms/step - loss: 1.0559 - accura

In [17]:
test_iter = iter(ds_test.batch(256))
x_test = []
y_test = []

for i in range(40 ):
  data = next(test_iter)
  x_test.append(data['image'])
  y_test.append(data['label'])
x_test = np.concatenate(x_test,axis=0)
x_test = x_test/255
y_test = np.concatenate(y_test,axis=0)
one_hot = np.zeros((len(y_test),10))
one_hot[np.arange(len(y_test)),y_test] = 1
y_test = one_hot 



In [18]:
model.evaluate(x_test,y_test,batch_size=256)

40/40 [==============================] - 0s 5ms/step - loss: 2.6517 - accuracy: 0.5976


[2.651672840118408, 0.597599983215332]

# save a model

# load a model

## Evaluate complexity measure

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !python '/content/drive/My Drive/PGDL/submission files/ingestion_program/ingestion.py' '/content/drive/My Drive/PGDL/submission files/sample_data' '/content/drive/My Drive/PGDL/submission files/sample_result_submission'  '/content/drive/My Drive/PGDL/submission files/ingestion_program/' '/content/margin_normalised_path_norm.zip'

In [1]:
# !python '/content/drive/My Drive/PGDL/submission files/scoring_program/score.py' '/content/drive/My Drive/PGDL/submission files/sample_data' '/content/drive/My Drive/PGDL/submission files/sample_result_submission' '/content/drive/My Drive/PGDL/submission files/sample_score'
